In [3]:
import os
import librosa
import numpy as np
from tqdm import tqdm
import soundfile as sf  # Import soundfile module
from itertools import combinations
from audiomentations import Compose, PitchShift, TimeStretch, AddGaussianNoise, Gain

# Define the input and output directories
input_dir = r'augmented'
output_dir = r'after augmentation'

# Define the target number of samples for each class
target_samples = 2000

# Define augmentation techniques
augmentations = [
    PitchShift(min_semitones=-4, max_semitones=4, p=0.5),
    TimeStretch(min_rate=0.8, max_rate=1.2, p=0.5),
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
    Gain(min_gain_in_db=-6, max_gain_in_db=6, p=0.5)
]

# Function to perform augmentation and save audio
def augment_and_save(audio_path, class_output_dir, filename, augmentation):
    y, sr = librosa.load(audio_path, sr=None)
    augmented_y = augmentation(samples=y, sample_rate=sr)
    output_filename = os.path.join(class_output_dir, f"{os.path.splitext(filename)[0]}_{augmentation.__class__.__name__}_aug.wav")
    sf.write(output_filename, augmented_y, sr)

# Function to perform combination of augmentations and save audio
def combine_and_save(audio_path, class_output_dir, filename, augmentations):
    y, sr = librosa.load(audio_path, sr=None)
    augmented_y = y
    for augmentation in augmentations:
        augmented_y = augmentation(samples=augmented_y, sample_rate=sr)
    combo_names = '_'.join([aug.__class__.__name__ for aug in augmentations])
    output_filename = os.path.join(class_output_dir, f"{os.path.splitext(filename)[0]}_{combo_names}_aug.wav")
    sf.write(output_filename, augmented_y, sr)

# Iterate through each class
classes = os.listdir(input_dir)


for class_name in classes:
    # Create output directory if it doesn't exist
    class_output_dir = os.path.join(output_dir, class_name)
    os.makedirs(class_output_dir, exist_ok=True)
    
    # Count the number of existing samples
    existing_samples = len(os.listdir(class_output_dir))
    
    # Check if the total number of samples is already more than 2000
    if existing_samples >= target_samples:
        print(f"Class '{class_name}' already has enough samples.")
        continue
    
    # Iterate through each audio file
    for filename in tqdm(os.listdir(os.path.join(input_dir, class_name)), desc=f"Augmenting {class_name}"):
        audio_path = os.path.join(input_dir, class_name, filename)
        
        # Skip augmentation if already enough samples
        if existing_samples >= target_samples:
            break
        
        # Perform single augmentations
        for augmentation in augmentations:
            augment_and_save(audio_path, class_output_dir, filename, augmentation)
            existing_samples += 1
            if existing_samples >= target_samples:
                break
                
        if existing_samples >= target_samples:
            break
                
        # Perform combination of two augmentations
        for combo in combinations(augmentations, 2):
            combine_and_save(audio_path, class_output_dir, filename, combo)
            existing_samples += 1
            if existing_samples >= target_samples:
                break
                
        if existing_samples >= target_samples:
            break
                
        # Perform combination of three augmentations
        for combo in combinations(augmentations, 3):
            combine_and_save(audio_path, class_output_dir, filename, combo)
            existing_samples += 1
            if existing_samples >= target_samples:
                break
                
        if existing_samples >= target_samples:
            break
                
        # Perform combination of four augmentations
        for combo in combinations(augmentations, 4):
            combine_and_save(audio_path, class_output_dir, filename, combo)
            existing_samples += 1
            if existing_samples >= target_samples:
                break
                
        if existing_samples >= target_samples:
            break


Augmenting book_page_flip:   0%|          | 0/124 [00:00<?, ?it/s]

Augmenting book_page_flip: 100%|██████████| 124/124 [03:28<00:00,  1.68s/it]
